In [2]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-04-30 15:58:10--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  27.7MB/s    in 4m 7s   

2022-04-30 16:02:18 (26.0 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-04-30 16:02:49--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [57]:
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt

--2022-04-30 04:10:35--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt.1’

map_clsloc.txt.1    100%[===================>]  23.79K  --.-KB/s    in 0.002s  

2022-04-30 04:10:36 (11.6 MB/s) - ‘map_clsloc.txt.1’ saved [24366/24366]



In [3]:
from torchvision import models
import torch
from PIL import Image

In [12]:
mobilenet_v2 = models.mobilenet_v2(pretrained=True)
mobilenet_v2.eval()

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

MobileNetV2(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05,

In [5]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [6]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')
validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
classes_mapping_list = []
classes_mapping = {}
for i in range(len(mapping_list_from)):
  classes_mapping[mapping_list_from[i].split(" ")[2].replace('\n',"")] = mapping_list_from[i].split(" ")[1]
for i in range(len(classes)):
  classes_mapping_list.append(classes_mapping[classes[i].split(", ")[1]])

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

In [13]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  # print(img_path)
  # img = Image.open("dog.jpg")
  img = Image.open(img_path).convert('RGB')
  img_t = transform(img)
  batch_t = torch.unsqueeze(img_t, 0)

  out = mobilenet_v2(batch_t)

  _, index = torch.topk(input=out,k=5)

  # print(index[0].tolist())

  if classes_mapping_list[index[0][0]] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      if classes_mapping_list[index[0][ii]] == validation_list[i]:
        top_5 = top_5 + 1
        break
  # print(correct, top_5)
  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

1%, Top 1:0.7 Top 5:0.902
2%, Top 1:0.709 Top 5:0.905
3%, Top 1:0.728 Top 5:0.902
4%, Top 1:0.724 Top 5:0.8985
5%, Top 1:0.7292 Top 5:0.902
6%, Top 1:0.7283333333333334 Top 5:0.9013333333333333
7%, Top 1:0.7271428571428571 Top 5:0.9028571428571428
8%, Top 1:0.726 Top 5:0.9015
9%, Top 1:0.7262222222222222 Top 5:0.9013333333333333
10%, Top 1:0.7256 Top 5:0.8992
11%, Top 1:0.7258181818181818 Top 5:0.8983636363636364
12%, Top 1:0.725 Top 5:0.8996666666666666
13%, Top 1:0.726 Top 5:0.8984615384615384
14%, Top 1:0.7252857142857143 Top 5:0.8981428571428571
15%, Top 1:0.7253333333333334 Top 5:0.898
16%, Top 1:0.72375 Top 5:0.898
17%, Top 1:0.7229411764705882 Top 5:0.8977647058823529
18%, Top 1:0.7228888888888889 Top 5:0.898
19%, Top 1:0.7249473684210527 Top 5:0.8985263157894737
20%, Top 1:0.7216 Top 5:0.8985
21%, Top 1:0.7211428571428572 Top 5:0.8987619047619048
22%, Top 1:0.7204545454545455 Top 5:0.8984545454545455
23%, Top 1:0.7200869565217392 Top 5:0.8981739130434783
24%, Top 1:0.7201666666

In [14]:
accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

Top 1 Accuracy:71.804%
Top 5 Accuracy:90.102%
